In [8]:
import json
from pprint import pprint
import requests

# Yandex translate

    https://translate.yandex.net/api/v1.5/tr.json/translate
     ? [key=<API-ключ>]
     & [text=<переводимый текст>]
     & [lang=<направление перевода>]
     & [format=<формат текста>]
     & [options=<опции перевода>]
     & [callback=<имя callback-функции>]

In [9]:
API_KEY = 'trnsl.1.1.20171220T114400Z.d11d73895b5b665a.88eb58e16dfb38390744de068580cdf9001f0ec9'
request_json = {'key':API_KEY,'text': "Привет, мир!", 'lang':'ru-en'}
url = "https://translate.yandex.net/api/v1.5/tr.json/translate"

In [10]:
r = requests.post(url, data=request_json)
result = json.loads(s=r.text)
print(r.status_code, r.reason)
print(result['text'])

200 OK
['Hello world!']


In [72]:
def get_translation(phrase):
    req_json = {'key':API_KEY,'text': phrase, 'lang':'ru-en'}
    r = requests.post(url, data=req_json)
    result = json.loads(s=r.text)
    if r.status_code == 200:
        return result['text']
    else:
        return -1

In [73]:
get_translation('Я люблю горные пейзажи и белок на деревьях')

['I love mountain scenery and squirrels in the trees']

# Yandex Dict

In [12]:
DICT_API_KEY = 'dict.1.1.20171220T134405Z.0ce08af556412b54.d0564c80bbf2c07080c970656db59eddc61e0fbc'
dict_url = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup'

In [192]:
def get_json(word):
    dict_request_json = {'key': DICT_API_KEY, 'text':word, 'lang':'ru-en'}
    r = requests.post(dict_url, data=dict_request_json)
    result = json.loads(s=r.text)
#     if r.status_code == 200:
#         pprint(result)
        
    means = []
    for definit in result['def']:
        for tr in definit['tr']:
            if 'syn' in tr:
                tmp = [tr['text']]
                tmp.extend([syn['text'] for syn in tr['syn']])
                means.append(tmp)
            else:
                means.append(tr['text'])
    return result, means

In [97]:
from pymystem3 import Mystem
mystem = Mystem()
mystem.lemmatize('банок')


['банка', '\n']

In [193]:
r, means = get_json('банк')
print(means)

[['bank', 'banker', 'banking'], ['pot', 'jar'], 'jackpot']


In [194]:
r, means = get_json('банка')
print(means)

[['bank', 'jar', 'pot'], 'bench']


In [195]:
r, means = get_json('бить')
print(means)

[['beat', 'strike', 'smite', 'hit', 'punch', 'slap', 'bang', 'knock', 'smack', 'whack'], ['flog', 'trounce'], 'break', 'hurt', 'kill', 'pound', 'go', 'bash', 'hammer', 'club', 'thrash', 'wallop', 'pummel', 'baste', 'whale']


# Experiment with wiki-wiki test

In [69]:
from tqdm import tqdm_notebook

In [196]:
from testing_interface import make_data,\
                              visualize_pca,\
                              evaluate_weighted_ari,\
                              get_one_word_data,\
                              visualize_pca_one_word,\
                              visualize_tsne_one_word

DATASET = '../data/main/wiki-wiki/test.csv'

contexts, main_words, labels, word_list_uniq = make_data(df_file_name=DATASET,\
                 use_gensim_simple_preproc=True,\
                 throw_main_word=False, tokenize=True)

In [197]:
word_list_uniq

array(['банка', 'белка', 'бит', 'горе', 'гранат', 'граф', 'душ'], dtype=object)

In [198]:
contexts[:3]

array([ list(['здание', 'банка', 'китая', 'макао', 'здание', 'банка', 'китая', 'bank', 'of', 'china', 'building', 'edificio', 'banca', 'da', 'china', '澳门中银大厦', 'этажный', 'небоскреб', 'высотой', 'метра', 'расположенный', 'макао', 'местная', 'штаб', 'квартира', 'bank', 'of', 'china', 'построен']),
       list(['трехлитровая', 'банка', 'во', 'времена', 'ссср', 'такие', 'банки', 'были', 'популярны', 'для', 'маринованных', 'овощей', 'овощных', 'фруктовых', 'соков', 'так', 'далее', 'популярность', 'трехлитровых', 'банок', 'объясняется', 'тем', 'что', 'это', 'самая', 'объемная', 'банка', 'из', 'массово', 'доступных', 'это', 'удобно', 'при', 'большом', 'объеме', 'заготовок', 'наши', 'дни', 'стеклянные', 'банки', 'продолжают', 'использоваться', 'быту', 'для', 'домашнего', 'консервирования', 'подготовка', 'абсолютно', 'целых', 'без', 'трещин', 'сколов', 'стеклянных', 'банок', 'подразумевает', 'не', 'только', 'тщательное', 'мытье', 'внутри', 'снаружи', 'слабым']),
       list(['здание', 'национа

In [199]:
def get_translations_list(contexts):
    translations = []
    for c in tqdm_notebook(contexts):
        tr = get_translation(' '.join(c))
        translations.append(tr)
    return np.array(translations)

In [89]:
wiki_transl = get_translations_list(contexts=contexts)

HBox(children=(IntProgress(value=0, max=638), HTML(value='')))

In [200]:
wiki_transl[:4]

[['building Bank of China Macau Bank of China building bank of china building, edificio banca da china 澳门中银大厦 storey skyscraper with a height of meters, Macau, the local headquarters of bank of china built'],
 ['three-liter jar during Soviet times, these banks have been popular for pickled vegetables vegetable fruit juices so on the popular three-liter cans due to the fact that it is the volume of the Bank available in large quantities is useful when a large amount of pieces of our days glass jars continue to be used everyday for home canning training is completely whole with no cracks chips of glass jars involves not only a thorough wash out the weak'],
 ['the building of the national Bank of Ukraine the national Bank of Ukraine is the office of the national Bank of Ukraine is the historical area of Pechersk Lypky district of Kiev at Institutskaya street office of the commercial Bank of Russia in Kiev was founded more than a century and a half ago year with time for her was bought a l

In [91]:
[' '.join(c) for c in contexts[:4]]

['здание банка китая макао здание банка китая bank of china building edificio banca da china 澳门中银大厦 этажный небоскреб высотой метра расположенный макао местная штаб квартира bank of china построен',
 'трехлитровая банка во времена ссср такие банки были популярны для маринованных овощей овощных фруктовых соков так далее популярность трехлитровых банок объясняется тем что это самая объемная банка из массово доступных это удобно при большом объеме заготовок наши дни стеклянные банки продолжают использоваться быту для домашнего консервирования подготовка абсолютно целых без трещин сколов стеклянных банок подразумевает не только тщательное мытье внутри снаружи слабым',
 'здание национального банка украины здании национального банка украины находится офис национального банка украины расположено исторической местности липки печерском районе города киева по адресу ул институтская контора коммерческого банка россии киеве была основана более полутора столетий тому назад году со временем для нее 

# Checking base theory

In [98]:
import pandas as pd

In [140]:
from Levenshtein import distance

In [204]:
df = pd.read_csv(DATASET, sep='\t')
df.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,440,банка,NaN,NaN,"7-12, 36-41",здание банка китая ( макао ) здание банка кита...
1,441,банка,NaN,NaN,"13-18, 201-206",трехлитровая банка во времена ссср такие банки...
2,442,банка,NaN,NaN,"21-26, 58-63, 101-106, 263-268, 538-543",здание национального банка украины в здании на...
3,443,банка,NaN,NaN,"134-139, 262-267, 345-350",зюдфиртель города эссен ( федеральная земля се...
4,444,банка,NaN,NaN,"43-48, 300-305, 381-386, 521-526",ведения боевых действий полевые учреждения бан...


In [205]:
len(df[df.word == 'белка'])

135

In [137]:
def parse_positions(pos: pd.Series, context: pd.Series):
    bag_of_words = []
    for p,c in zip(pos.values, context.values):
        positions = [n.split('-') for n in p.split(', ')]
        for s,f in positions:
            bag_of_words.append(c[int(s):int(f)])
    return set(bag_of_words)

In [139]:
parse_positions(df.positions, df.context)

{'банка', 'белка', 'бит', 'горе', 'гранат', 'граф', 'душ'}

In [325]:
def find_word_forms(context: pd.Series, word:str):
    w_forms = []
    if len(word)>3:
        for c in context:
            for w in c:
                if distance(w, word)==1:
                    w_forms.append(w)
        return set(w_forms)
    else:
        return set([word])

# test on wiki-wiki train

In [295]:
dataset = "../data/main/wiki-wiki/train.csv"
df = pd.read_csv(dataset, sep='\t')
df.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,NaN,замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,NaN,"шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,NaN,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,NaN,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,NaN,великий князь литовский гедимин после успешной...


In [296]:
contexts, main_words, labels, word_list_uniq = make_data(df_file_name=dataset,\
                 use_gensim_simple_preproc=True,\
                 throw_main_word=False, tokenize=True)

In [297]:
wiki_transl_train = get_translations_list(contexts=contexts)

HBox(children=(IntProgress(value=0, max=439), HTML(value='')))

In [305]:
word_list_uniq

array(['замок', 'лук', 'суда', 'бор'], dtype=object)

In [298]:
def find_all_meanings(contexts, word):
    w_forms = list(find_word_forms(context=contexts, word=word))
    stemmed = [mystem.lemmatize(w)[0] for w in w_forms]
    w_forms.append(word)
    w_forms.extend(stemmed)
    w_forms = list(set(w_forms))
    meanings = []
    for w in w_forms:
        r, means = get_json(w)
        meanings.extend(means)
    return meanings

In [301]:
from collections import defaultdict

In [331]:
import numpy as np

def get_markers(m, size):
    mapping = {}
    markers = np.zeros(size)
    c = 1
    for i, val in m.items():
        val = tuple(val)
        if val not in mapping:
            mapping[val]=c
            c+=1
        markers[i]=mapping[val]
    return markers

In [332]:
def find_meanings_in_contexts(contexts, translation, word):
    meanings = find_all_meanings(contexts=contexts, word=word)
    print(meanings)
    marks = defaultdict(list) 
    for i,c in enumerate(translation):
        flag = False
        c = c[0].split()
        for m in meanings:
            if type(m) == str:
                if m in c:
                    marks[i].append(m)
                    flag = True
            else:
                for t in m:
                    if t in c:
                        flag = True
                        marks[i].append(m[0])
                    break
    markers = get_markers(marks, len(contexts))
    return markers

In [333]:
resulted_markup = []

for word in word_list_uniq:
    marks = find_meanings_in_contexts(contexts[main_words == word],
                                      wiki_transl_train[main_words == word], word)
    resulted_markup.extend(marks)

[['castle', 'Chateau'], ['lock', 'padlock', 'latch']]
['onion', 'bow']
[['court', 'tribunal', 'trial', 'courtroom'], ['judgment', 'judgement'], 'justice', ['craft', 'boat'], 'law', 'forum', 'bench', ['vessel', 'boat', 'craft', 'watercraft', 'ship'], 'hovercraft', 'cruiser']
['boron', 'drill', 'bor', ['pine forest', 'coniferous forest'], 'pinery', 'milletgrass', 'Bohr']


In [337]:
evaluate_weighted_ari(dataset, resulted_markup)

word	ari	count
бор	0.279725	56
замок	0.654670	138
лук	0.219787	110
суда	-0.008544	135
	0.293923	439
